In [1]:
import quandl
import pandas as pd
import numpy as np
import talib
import csv

In [2]:
gold=quandl.get("PERTH/GOLD_USD_D", authtoken="B5RSB-jK5vYvZKyFBm4q")
silver=quandl.get("PERTH/SLVR_USD_D", authtoken="B5RSB-jK5vYvZKyFBm4q")
oil=quandl.get("FRED/DCOILWTICO", authtoken="B5RSB-jK5vYvZKyFBm4q")
ruthenium=quandl.get("JOHNMATT/RUTH", authtoken="B5RSB-jK5vYvZKyFBm4q")
iridium=quandl.get("JOHNMATT/IRID", authtoken="B5RSB-jK5vYvZKyFBm4q")
rhodium=quandl.get("JOHNMATT/RHOD", authtoken="B5RSB-jK5vYvZKyFBm4q")
palladium=quandl.get("JOHNMATT/PALL", authtoken="B5RSB-jK5vYvZKyFBm4q")
platinum=quandl.get("JOHNMATT/PLAT", authtoken="B5RSB-jK5vYvZKyFBm4q")

In [3]:
oil=oil.rename(columns={"Value":'value'})

In [4]:
gold_value=pd.DataFrame((gold['Bid Average']+gold['Ask Average'])/2).rename(columns={0:'value'})
silver_value=pd.DataFrame((silver['Bid Average']+silver['Ask Average'])/2).rename(columns={0:'value_silver'})
ruth_value=pd.DataFrame(ruthenium['New York 9:30']).rename(columns={'New York 9:30':'value_ruth'})
irid_value=pd.DataFrame(iridium['New York 9:30']).rename(columns={'New York 9:30':'value_irid'})
rhod_value=pd.DataFrame(rhodium['New York 9:30']).rename(columns={'New York 9:30':'value_rhod'})
pall_value=pd.DataFrame(palladium['New York 9:30']).rename(columns={'New York 9:30':'value_pall'})
plat_value=pd.DataFrame(platinum['New York 9:30']).rename(columns={'New York 9:30':'value_plat'})
hard=gold_value.join(oil,how='outer',lsuffix='_gold',rsuffix='_oil').join(silver_value,how='outer').join(ruth_value,how='outer').join(irid_value,how='outer').join(pall_value,how='outer').join(rhod_value,how='outer').join(plat_value,how='outer').dropna()

In [5]:
hard

,value_gold,value_oil,value_silver,value_ruth,value_irid,value_pall,value_rhod,value_plat
Date,,,,,,,,
1992-07-01,343.400,21.90,4.045,32.0,200.0,83.0,2675.0,385.0
1992-07-02,344.300,22.08,4.050,32.0,200.0,84.0,2725.0,389.0
1992-07-03,346.130,22.08,4.060,32.0,200.0,84.0,2725.0,386.0
1992-07-06,346.250,21.87,4.060,32.0,200.0,84.0,2725.0,388.0
1992-07-07,346.550,21.51,4.060,32.0,200.0,83.0,2700.0,383.0
...,...,...,...,...,...,...,...,...
2020-03-24,1575.415,21.03,13.780,250.0,1510.0,1932.0,5500.0,692.0
2020-03-25,1617.155,20.75,14.435,250.0,1510.0,2058.0,8250.0,724.0
2020-03-26,1605.405,16.60,14.320,250.0,1510.0,2322.0,10000.0,742.0


In [7]:
hard_tech=pd.DataFrame()
cols=hard.columns
for x in cols:
    hard_tech['macd'+x[x.find('_'):]]=talib.MACD(hard[x])[0]
    hard_tech['rsi'+x[x.find('_'):]]=talib.RSI(hard[x])

In [9]:
hard_tech.count()

macd_gold      6615
rsi_gold       6634
macd_oil       6615
rsi_oil        6634
macd_silver    6615
rsi_silver     6634
macd_ruth      6615
rsi_ruth       6634
macd_irid      6615
rsi_irid       6634
macd_pall      6615
rsi_pall       6634
macd_rhod      6615
rsi_rhod       6634
macd_plat      6615
rsi_plat       6634
dtype: int64

In [10]:
hard_tech=hard_tech.dropna()

In [12]:
hard_pct=pd.DataFrame()
cols=hard.columns
for x in cols:
    hard_pct['pct'+x[x.find('_'):]]=hard[x].pct_change()

In [16]:
hard_pct=hard_pct.replace([np.inf, -np.inf,np.nan], 0)

In [18]:
hard_pct.count()

pct_gold      6648
pct_oil       6648
pct_silver    6648
pct_ruth      6648
pct_irid      6648
pct_pall      6648
pct_rhod      6648
pct_plat      6648
dtype: int64